In [15]:
import os
import struct
import rawutil
import numpy as np

aux = np.load(os.path.join('data', 'AUX.npy'))
eeg = np.load(os.path.join('data', 'EEG.npy'))

# Deserialization

## AUX

### ```rawutil```

In [29]:
%%timeit
np.array([[rawutil.unpack('>H', bytes(ch))[0] for ch in row.reshape(-1, 2).tolist()][:2] for row in aux])

2.36 ms ± 89.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### ```struct```

In [30]:
%%timeit
np.array([struct.unpack('>hhh', a.astype('i1').tobytes())[:2] for a in aux])

185 µs ± 3.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## EEG

### ```rawutil```

In [130]:
%%timeit
np.array([[rawutil.unpack('>u', bytes(ch))[0] for ch in row.reshape(-1, 3).tolist()] for row in eeg])

6.23 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### ```struct```

In [131]:
%%timeit
np.array([struct.unpack('>i', (b'\0' if chunk[0] < 128 else b'\xff') + chunk) for chunk in [bytes(ch.tolist()) for ch in eeg.reshape(-1, 3)]]).reshape(-1, 8)

1.11 ms ± 35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
